In [27]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import sys

sys.path.append('./src/jubilee_pipette_bodemo/')
sys.path.append('../bayesopt')

from science_jubilee import Machine as Jub
from science_jubilee.labware.Labware import Labware, Well
from science_jubilee.tools import Pipette, WebCamera, Tool

import color_match
from color_match import BO_campaign
import jubilee_protocols

from bayesopt import acquisitions
from bayesopt import bayesian_optimizer

import ipywidgets as widgets
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from math import sqrt, acos, asin, cos, sin
import copy

import json

## Machine setup and connection

In [4]:
jubilee = Jub.Machine(address='192.168.1.2')

In [5]:
jubilee.home_all()

In [6]:
jubilee.move_to(z = 115)

In [7]:
deck = jubilee.load_deck('lab_automation_deck.json')
tiprack = jubilee.load_labware('opentrons_96_tiprack_300ul.json', 0)
samples = jubilee.load_labware('corning_96_wellplate_360ul_flat.json', 2)
stocks = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 3)
trash = jubilee.load_labware('agilent_1_reservoir_290ml.json', 1)

In [8]:
#configure pipette
P300 = Pipette.Pipette.from_config(1, 'Pipette', 'P300_config.json')
jubilee.load_tool(P300)
P300.add_tiprack(tiprack)

In [9]:
#configure camera
Camera = WebCamera.Camera.from_config(0, 'Camera', 'WebCamera_config.json')
jubilee.load_tool(Camera)

In [10]:
# apply offset to samples for mixer module 
#samples.offset = (0, 0, 40.0)

### Manual offset input

In [11]:
# tiprack
UL = [30.0,79.0]
UR = [129.0, 79.4]
BR = [129.5, 16.9]

tiprack.manual_offset((UL, UR, BR), save = True) #, save=True, force=True)

New manual offset applied to opentrons_96_tiprack_300ul
Manual offset saved


In [12]:
# samples
UL = [32.2,176.4]
UR = [123.3, 176.9]
BR = [123.6, 112.6]

samples.manual_offset((UL, UR, BR), save=True)

New manual offset applied to corning_96_wellplate_360ul_flat
Manual offset saved


In [13]:
# stocks
UL = [175.5, 171.5]
UR = [259.3, 172.1]
BR = [259.9, 117.0]

stocks.manual_offset((UL, UR, BR), save=True)

New manual offset applied to 20mlscintillation_12_wellplate_18000ul
Manual offset saved


## Color picking

In [14]:
a = widgets.ColorPicker(
    concise=False,
    description='Pick a color',
    value='blue',
    disabled=False
)

In [15]:
a

ColorPicker(value='blue', description='Pick a color')

In [16]:
hexcode = a.value


In [17]:
target_rgb = tuple(int(hexcode.lstrip('#')[i:i+2], 16) for i in (0,2,4))
target_color = color_match.normalize_color(target_rgb)

In [18]:
target_color

[0.5803921568627451, 0.12549019607843137, 0.5725490196078431]

## Set up things for demo

In [19]:
red_stock = stocks[0]
yellow_stock = stocks[2]
blue_stock = stocks[1]
trash_well = trash[0]

In [20]:
sample_space = color_match.get_constrained_points(101)

## Get initial data

In [28]:
n_init_points = 6
sampled_indices = np.random.randint(0, len(sample_space), size = n_init_points)
sampled_points = sample_space[sampled_indices]

In [19]:
rgb_vals = []
for i in range(len(sampled_points)):
    point = sampled_points[i,:]
    well = samples[i]
    print('creating sample')
    print(point)
    print(well)
    
    RGB = jubilee_protocols.sample_point(jubilee, P300, Camera, point, 250, well, red_stock, yellow_stock, blue_stock, trash_well)
    rgb_vals.append(RGB)

creating sample
[0.31 0.66 0.03]
Well(name='A1', depth=10.67, totalLiquidVolume=360, shape='circular', diameter=6.86, xDimension=None, yDimension=None, x=28.68, y=175.04, z=3.55, offset=[14.3, 100.8], slot=2)
Start of sample sequence position:  {'X': '283.500', 'Y': '305.000', 'Z': '115.000', 'U': '0.000', 'V': '0.500', 'E': '0.000'}
Calculated volumes:  [77.5, 165.0, 7.5]
creating sample
[0.33 0.55 0.12]
Well(name='A2', depth=10.67, totalLiquidVolume=360, shape='circular', diameter=6.86, xDimension=None, yDimension=None, x=37.68, y=175.04, z=3.55, offset=[14.3, 100.8], slot=2)
Start of sample sequence position:  {'X': '283.500', 'Y': '305.000', 'Z': '186.490', 'U': '0.000', 'V': '310.000', 'E': '0.000'}
Calculated volumes:  [82.5, 137.5, 30.0]
creating sample
[0.61 0.05 0.34]
Well(name='A3', depth=10.67, totalLiquidVolume=360, shape='circular', diameter=6.86, xDimension=None, yDimension=None, x=46.68000000000001, y=175.04, z=3.55, offset=[14.3, 100.8], slot=2)
Start of sample sequence

In [20]:
init_data = []
for i in range(len(sampled_points)):
    data = {}
    data['sample_id'] = str(i)
    data['RYB_point'] = list(sampled_points[i])
    data['RGB_measured'] = rgb_vals[i]

    init_data.append(data)

In [24]:
with open('initial_data_random_6.jsonl', 'wt') as f:
    for entry in init_data:
        f.write(json.dumps(entry) + '\n')

## Run BO campaign to make target color 

### Load initial data and calculate loss value for target color 

In [21]:
init_data = []

with open('initial_data_random_6.jsonl', 'rt') as f:
    for line in f:
        init_data.append(json.loads(line))

In [30]:
for entry in init_data:
    bgr = entry['RGB_measured']
    entry['RGB_measured'] = [bgr[i] for i in [2,1,0]]

In [32]:
init_X = np.zeros((len(init_data),3))
init_y = np.zeros(len(init_data))

for i, entry in enumerate(init_data):
    init_X[i,:] = entry['RYB_point']
    init_y[i] = color_match.color_loss_calculation(target_color, color_match.normalize_color(entry['RGB_measured']))

### Select acquisition function

In [33]:
acquisition_function = acquisitions.optimize_EI
acq_kwargs = {'xi':0.3}

In [34]:
initial_data = (init_X, init_y)
number_of_iterations = 2

In [35]:
ryb_sampled, rgb_measured, images, scores, bo_obj = BO_campaign(initial_data, acquisition_function, acq_kwargs, number_of_iterations, target_color, jubilee, P300, Camera, 250, stocks['A1'], stocks['A3'], stocks['A2'], samples, trash[0], start_well = 0)

Starting iteration 0
Dispensing into well Well(name='A1', depth=10.67, totalLiquidVolume=360, shape='circular', diameter=6.86, xDimension=None, yDimension=None, x=27.790954788973195, y=175.1833520398478, z=3.55, offset=[14.3, 100.8], slot=2)
RYB values tested: [0. 0. 1.]
Calculated volumes:  [0.0, 0.0, 250.0]
RGB values observed: {RGB}
Starting iteration 1
Dispensing into well Well(name='A2', depth=10.67, totalLiquidVolume=360, shape='circular', diameter=6.86, xDimension=None, yDimension=None, x=36.790838797196834, y=175.22904491150624, z=3.55, offset=[14.3, 100.8], slot=2)
RYB values tested: [1. 0. 0.]
Calculated volumes:  [250.0, 0.0, 0.0]
RGB values observed: {RGB}


In [38]:
len(samples.wells)

96